In [40]:
import numpy as np
import pandas as pd 

pd.options.display.float_format = '{:.2f}'.format
# Show all rows
pd.set_option('display.max_rows', None)

# Show all columns
pd.set_option('display.max_columns', None)

filename = "experiment_result_cart_pole_implicit_theory_hard_threshold_sparse_regression_sindy"

folder = "single_test_result"

ideal_solution = np.load(f"{folder}/{filename}_ideal_solution.npy")
raw_solution = np.load(f"{folder}/{filename}.npy")

pd.DataFrame(raw_solution)


,0,1,2,3,4,5,6,7,8
0,-1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,-1.00,0.00,-1.60,-0.62,0.00,0.00,1.60,0.00
2,0.00,0.00,-1.00,0.00,0.00,2.60,-2.60,0.00,0.27
3,0.00,-0.63,0.00,-1.00,-0.38,0.00,0.00,1.00,0.00
4,0.00,-1.62,0.00,-2.60,-1.00,0.00,0.00,2.60,0.00
5,0.00,0.00,0.38,0.00,0.00,-1.00,1.00,0.00,-0.10
6,0.00,0.00,-0.38,0.00,0.00,1.00,-1.00,0.00,0.10
7,0.00,0.62,0.00,1.00,0.38,0.00,0.00,-1.00,0.00
8,0.00,0.00,3.77,0.00,0.00,-9.81,9.81,0.00,-1.00


In [ ]:

raw_solution = raw_solution[1:,1:]
ideal_solution = ideal_solution[1:]

solution = - raw_solution @ np.diag(np.where(ideal_solution !=0 , ideal_solution, 1).flatten())

column_name = [ "in" if i!= 0 else "out" for i in ideal_solution.flatten() ]


In [32]:
print("Ideal solution:")
pd.DataFrame(ideal_solution)

Ideal solution:


,0
0,-0.80
1,-1.30
2,-0.50
3,-1.30
4,0.50
5,-0.50
6,0.50
7,4.91


In [33]:
print("Solution:")
pd.DataFrame(solution)#,columns=column_name)

Solution:


,0,1,2,3,4,5,6,7
0,-0.80,0.00,-0.80,-0.80,0.00,0.00,-0.80,0.00
1,0.00,-1.30,0.00,0.00,-1.30,-1.30,0.00,-1.30
2,-0.50,0.00,-0.50,-0.50,0.00,0.00,-0.50,0.00
3,-1.30,0.00,-1.30,-1.30,0.00,0.00,-1.30,0.00
4,0.00,0.50,0.00,0.00,0.50,0.50,0.00,0.50
5,0.00,-0.50,0.00,0.00,-0.50,-0.50,0.00,-0.50
6,0.50,0.00,0.50,0.50,0.00,0.00,0.50,0.00
7,0.00,4.90,0.00,0.00,4.91,4.91,0.00,4.91


In [34]:
## Study of difference between ideal solution and solution
diff = np.abs(ideal_solution - solution)/np.where(ideal_solution != 0, ideal_solution, 1)

pd.DataFrame(diff,columns=column_name)

,in,in,in,in,in,in,in,in
0,-0.00,-1.00,-0.00,-0.00,-1.00,-1.00,-0.00,-1.00
1,-1.00,-0.00,-1.00,-1.00,-0.00,-0.00,-1.00,-0.00
2,-0.00,-1.00,-0.00,-0.00,-1.00,-1.00,-0.00,-1.00
3,-0.00,-1.00,-0.00,-0.00,-1.00,-1.00,-0.00,-1.00
4,1.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00
5,-1.00,-0.00,-1.00,-1.00,-0.00,-0.00,-1.00,-0.00
6,0.00,1.00,0.00,0.00,1.00,1.00,0.00,1.00
7,1.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00


In [35]:
pd.DataFrame(np.abs(diff).mean(axis=0).reshape(1,-1),columns=column_name)

,in,in,in,in,in,in,in,in
0,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50


In [36]:
raw_solution_norm = np.linalg.norm(raw_solution, axis=0)
# Avoid divide-by-zero: keep zeros as zeros
raw_solution_norm[raw_solution_norm == 0] = 1.0
raw_solution = raw_solution / raw_solution_norm

In [37]:
# Asssumption 3 using Leiden algorithm to cluster thanks to cosine similarities

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import igraph as ig
import leidenalg as la

def leiden_cosine_clustering(data, k_max=None, min_cluster_size=2):
    """
    Clusters column vectors in `data` using the Leiden algorithm based on cosine similarity.

    Parameters:
    - data: np.ndarray of shape (n_features, n_vectors)
    - k_max: int, optional, maximum number of clusters to keep
    - min_cluster_size: int, minimum number of points required to keep a cluster

    Returns:
    - labels: np.ndarray of shape (n_vectors,), where -1 indicates discarded/noisy points
    """
    # Transpose to get shape (n_vectors, n_features)
    data = data.T
    n = data.shape[0]

    # Step 1: Cosine similarity matrix
    cosine_sim = np.abs(cosine_similarity(data))
    np.fill_diagonal(cosine_sim, 0)  # remove self-loops

    # Step 2: Build igraph from similarity matrix
    sources, targets = np.where(cosine_sim > 0)
    weights = cosine_sim[sources, targets]
    edges = list(zip(sources.tolist(), targets.tolist()))

    g = ig.Graph(edges=edges, directed=False)
    g.es['weight'] = weights

    # Step 3: Apply Leiden algorithm
    partition = la.find_partition(g, la.RBConfigurationVertexPartition, weights=g.es['weight'])

    # Step 4: Map nodes to clusters
    cluster_assignments = np.array(partition.membership)

    # Step 5: Filter small or weak clusters
    final_labels = -1 * np.ones(n, dtype=int)  # -1 means unassigned
    cluster_sizes = np.bincount(cluster_assignments)

    # Keep only clusters that are large enough
    valid_clusters = np.where(cluster_sizes >= min_cluster_size)[0]

    cluster_id_map = {old: new for new, old in enumerate(valid_clusters)}

    for i, cid in enumerate(cluster_assignments):
        if cid in valid_clusters:
            final_labels[i] = cluster_id_map[cid]

    # Step 6: Limit to k_max clusters (optional)
    if k_max is not None:
        unique_clusters = np.unique(final_labels[final_labels != -1])
        if len(unique_clusters) > k_max:
            # Rank clusters by size
            cluster_sizes = [(c, np.sum(final_labels == c)) for c in unique_clusters]
            top_clusters = sorted(cluster_sizes, key=lambda x: -x[1])[:k_max]
            top_cluster_ids = {c for c, _ in top_clusters}

            final_labels = np.array([l if l in top_cluster_ids else -1 for l in final_labels])

    return final_labels


labels = leiden_cosine_clustering(raw_solution, k_max=5)


pd.DataFrame(labels.reshape(1,-1),columns=column_name)

ModuleNotFoundError: No module named 'igraph'

In [ ]:
## Assumption n2 using group of close angle 

def groups_homothetic_noisy(mat, deg_tol=3.0, atol=1e-12):
    """
    mat  : shape (m, n) – columns are the vectors
    deg_tol : angular tolerance in degrees
    Returns a list of lists with column indices that are homothetic
    """
    rad_tol = np.deg2rad(deg_tol)
    # 1) normalise columns (keep track of zeros separately)
    lengths = np.linalg.norm(mat, axis=0)
    keep   = np.where(lengths > atol)[0]
    if keep.size == 0:
        return []                     # nothing but zeros
    unit   = mat[:, keep] / lengths[keep]

    groups, pool = [], list(range(unit.shape[1]))
    while pool:
        j        = pool.pop(0)
        ref      = unit[:, j]
        hits     = [keep[j]]          # column index in the original matrix
        to_drop  = []
        # 2) compare with remaining columns
        for pos in pool:
            ang = np.arccos(
                np.clip(np.abs(ref @ unit[:, pos]), -1.0, 1.0)
            )
            if ang <= rad_tol:
                hits.append(keep[pos])
                to_drop.append(pos)
        # 3) remove the ones we already grouped
        for pos in reversed(to_drop):
            pool.remove(pos)
        groups.append(hits)
    return groups

groups = groups_homothetic_noisy(raw_solution, deg_tol=15.0, atol=1e-12)

#Where is the mass of the vector in the group !?

def weak_sparsity_rank_weighted(x):
    """
    Custom weak sparsity measure: weighted average of sorted abs(x) using inverse rank weights.

    Parameters:
    - x: np.ndarray (1D vector)

    Returns:
    - scalar value ∈ [0, 1], higher = more concentrated mass at top
    """
    x = np.abs(x)
    s = np.sort(x)[::-1]  # descending
    ranks = np.arange(1, len(s) + 1)

    weights = 1.0 / ranks
    weighted_sum = np.sum(s * weights)
    weight_total = np.sum(weights)

    return weighted_sum / weight_total if weight_total != 0 else 0.0


print("Groups of homothetic vectors:")

filtered_groups = []

for i, group in enumerate(groups):
    if len(group) >= 3:

        total_group = raw_solution[:,group].sum(axis=1)
        weight= weak_sparsity_rank_weighted(total_group)

        if weight>0.7: 
            filtered_groups+= [group]

        print(f"Group {i}, weight {weight:.2f}: {group}")

candidate_function_2 =  groups[0]

print(filtered_groups)

print(np.argwhere(ideal_solution.flatten()!=0))

Groups of homothetic vectors:
Group 0, weight 0.87: [np.int64(0), np.int64(2), np.int64(3), np.int64(6)]
Group 1, weight 0.00: [np.int64(1), np.int64(4), np.int64(5), np.int64(7)]
[[np.int64(0), np.int64(2), np.int64(3), np.int64(6)]]
[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]]


In [ ]:
# normalize column 

treshold = 0.001

hard_threshold = np.where(np.abs(raw_solution) > treshold, solution, 0)

sparsity = np.where(hard_threshold !=0 ,1 ,0).sum(axis=0)

pd.DataFrame(sparsity.reshape(1,-1),columns=column_name)

In [ ]:
# sparsity calculation and decision 
decision = np.zeros((sparsity.shape[0]+1))
for j in sparsity:
    decision[j] += 1 

from matplotlib import pyplot as plt
# Validation cone for the sparsity 

cone_treshold = 1.5

plt.figure(figsize=(10, 5))
plt.bar(np.arange(len(decision)), decision, color='blue', alpha=0.7, label='Sparsity')
plt.plot(np.arange(len(decision)), np.arange(len(decision))*(1*cone_treshold), color='red', alpha=0.7, label='Decision')
plt.plot(np.arange(len(decision)), np.arange(len(decision))*(1/cone_treshold), color='red', alpha=0.7, label='Decision')
plt.show()

# This is clearly a test that is successful so I try to implement it in the main code

In [ ]:
# Assumption n1 sparsity cone 

passed_function = np.where((decision < np.arange(len(decision))*(1*cone_treshold)) &(decision > np.arange(len(decision))*(1/cone_treshold)), 1, 0)
sparsity_passed = np.flatnonzero(passed_function)

print("Sparsity passed:")
print(sparsity_passed)

candidate_function = np.isin(sparsity,sparsity_passed)
print("Candidate function:")

pd.DataFrame(candidate_function.reshape(1,-1),columns=column_name)
# Amazing it works !!! now I can combine the resultant into a single array

In [ ]:
#assumption n2
candidate_function = candidate_function_2


#candidate_solution = hard_threshold[np.ix_(candidate_function, candidate_function)]
candidate_solution = hard_threshold[:, candidate_function]


U, S, VT = np.linalg.svd(candidate_solution.T)

#retrieved_solution = np.zeros_like(ideal_solution)
#retrieved_solution[candidate_function,0] = -VT[0].flatten() 

retrieved_solution = -VT[0].reshape(-1,1) 

# print("Retrieved solution:")
# print(retrieved_solution.shape)

df = pd.DataFrame({
    'Retrieved Vector': retrieved_solution.flatten()/np.linalg.norm(retrieved_solution),
    'Ideal Vector': ideal_solution.flatten()/np.linalg.norm(ideal_solution),
})

df

In [ ]:
print("Solution difference:")
restricted_df = df[df['Ideal Vector'] != 0]

solution_difference =  np.abs(restricted_df['Retrieved Vector'] - restricted_df['Ideal Vector'])/np.abs(restricted_df['Ideal Vector'])

print(solution_difference.mean())
print("Solution difference max:")
print(solution_difference.max())
print("Solution difference min:")
print(solution_difference.min())
print("Solution difference std:")
print(solution_difference.std())
print("Solution difference median:")
print(solution_difference.median())

In [ ]:
#Other way to deal with group 

solutions = np.zeros((raw_solution.shape[0],len(filtered_groups)))

for i,group in enumerate(filtered_groups) :


    candidate_solution = raw_solution[:, group]


    U, S, VT = np.linalg.svd(candidate_solution.T)


    solutions[:,i] += -VT[0].flatten()

print(solutions.shape)

#This is a way to combine disjoint spaces in something comparable 
def combine_best_fit(solutions, v_ideal):
    """
    Return the best linear combination of a and b to fit v_ideal.

    Parameters:
    - solutions: disjoint solution array
    - v_ideal: target vector

    Returns:
    - v_hat: best fit vector (alpha * a + beta * b)
    - residual: L2 norm of the residual (||v_hat - v_ideal||)
    """

    x, _, _, _ = np.linalg.lstsq(solutions, v_ideal, rcond=None)
    v_hat = solutions @ x
    residual = np.linalg.norm(v_hat - v_ideal)
    return v_hat, residual

retrieved_solution ,_ = combine_best_fit(solutions,ideal_solution)

treshold=0.001 

print(np.linalg.norm(retrieved_solution))
retrieved_solution = np.where(np.abs(retrieved_solution)>treshold,retrieved_solution,0)

df = pd.DataFrame({
    'Retrieved Vector': retrieved_solution.flatten()/np.linalg.norm(retrieved_solution),
    'Ideal Vector': ideal_solution.flatten()/np.linalg.norm(ideal_solution),
})

df


In [ ]:

print("Solution difference:")
#restricted_df = df[df['Ideal Vector'] != 0]
normalise = np.where(df['Ideal Vector'] != 0,df['Ideal Vector'],1)

solution_difference =  np.abs(df['Retrieved Vector'] - df['Ideal Vector'])/np.abs(normalise)

print(solution_difference.mean())
print("Solution difference max:")
print(solution_difference.max())
print("Solution difference min:")
print(solution_difference.min())
print("Solution difference std:")
print(solution_difference.std())
print("Solution difference median:")
print(solution_difference.median())